# Singular Value Decomposition (SVD) Based Recommendation System Using Surprise Library

## Import necessary libraries

In [2]:

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import ndcg_score as NDCG


## Load the dataset

In [3]:
# Load the dataset
reader = Reader(line_format='user item rating timestamp', sep='::')
data = Dataset.load_from_file('ratings.dat', reader=reader)

## Data preprocessing: Spltting the dataset into 80% training and 20% testing

In [4]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

## Using GridSearchCV to find the best parameters

In [5]:
params = { "n_epochs":range(10,100,10), "lr_all":[0.002, 0.005], "reg_all":[0.02, 0.4, 0.6] }
grid_search = GridSearchCV(SVD, params, measures=["rmse", "mae"], refit=True, cv=5)
grid_search.fit(data)

training_parameters = grid_search.best_params["rmse"]
print("BEST RMSE: \t", grid_search.best_score["rmse"])
print("BEST params: \t", grid_search.best_params["rmse"])

BEST RMSE: 	 0.8734795898130537
BEST params: 	 {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [29]:
print("Best MAE score:", grid_search.best_score['mae'])
print("Best MAE parameters:", grid_search.best_params['mae'])

Best MAE score: 0.6858446360633019
Best MAE parameters: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


## Train the model using the best parameters and test the testing dataset

In [30]:
svd=SVD(n_epochs=training_parameters['n_epochs'], lr_all=training_parameters['lr_all'], reg_all=training_parameters['reg_all'])
svd.fit(trainset)

## Calculate precision, recall, ndcg, and f_score

In [31]:
def precision_recall_calculation(predictions, threshold=3.5):
    # First map the predictions to each user.
    user_predict_true = defaultdict(list)
    for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
        user_predict_true[user_id].append((predicted_rating, true_rating))

    precisions = dict()
    recalls = dict()
    for user_id, user_ratings in user_predict_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        no_of_relevant_items = sum((true_rating >= threshold) for (predicted_rating, true_rating) in user_ratings)
        # Number of recommended items in top 10
        no_of_recommended_items = sum((predicted_rating >= threshold) for (predicted_rating, true_rating) in user_ratings[:10])
        # Number of relevant and recommended items in top 10
        no_of_relevant_and_recommended_items = sum(((true_rating >= threshold) and (predicted_rating >= threshold)) for (predicted_rating, true_rating) in user_ratings[:10])
        # Precision: Proportion of recommended items that are relevant
        precisions[user_id] = no_of_relevant_and_recommended_items / no_of_recommended_items if no_of_recommended_items != 0 else 1
        # Recall: Proportion of relevant items that are recommended
        recalls[user_id] = no_of_relevant_and_recommended_items / no_of_relevant_items if no_of_relevant_items != 0 else 1

    # Averaging the values for all users
    average_precision=sum(precision for precision in precisions.values()) / len(precisions)
    average_recall=sum(recall for recall in recalls.values()) / len(recalls)
    F_score=(2*average_precision*average_recall) / (average_precision + average_recall)
    
    return [average_precision, average_recall, F_score]

In [33]:
test_set_predicted = svd.test(testset)

In [35]:
from collections import defaultdict

In [36]:
precision, recall, F_score = precision_recall_calculation(test_set_predicted, threshold=3.5)
print("Precision: \t{}".format(precision))
print("Recall: \t{}".format(recall))
print("F_score: \t{}".format(F_score))

Precision: 	0.8041355218257376
Recall: 	0.552559265793466
F_score: 	0.6550220986965127


In [6]:
# Print the best RMSE and MAE scores and the corresponding hyperparameters
print("Best RMSE score:", grid_search.best_score['rmse'])
print("Best RMSE parameters:", grid_search.best_params['rmse'])
print("Best MAE score:", grid_search.best_score['mae'])
print("Best MAE parameters:", grid_search.best_params['mae'])

Best RMSE score: 0.8520059180089629
Best RMSE parameters: {'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.06}
Best MAE score: 0.6717937085490093
Best MAE parameters: {'n_factors': 100, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.06}


In [7]:
# Train the algorithm with the best hyperparameters
best_algo = SVD(n_factors=grid_search.best_params['rmse']['n_factors'], n_epochs=grid_search.best_params['rmse']['n_epochs'], lr_all=grid_search.best_params['rmse']['lr_all'], reg_all=grid_search.best_params['rmse']['reg_all'])
best_algo.fit(trainset)

In [8]:
# Make predictions for the test set
predictions = best_algo.test(testset)

In [9]:

# Compute and print the RMSE and MAE scores for the test set
print("RMSE:", accuracy.rmse(predictions))
print("MAE:", accuracy.mae(predictions))


RMSE: 0.8524
RMSE: 0.8524218234875797
MAE:  0.6718
MAE: 0.6717813589487042


In [24]:
# Create empty lists for true and predicted ratings
true_ratings = []
pred_ratings = []

# Fill in the lists with true and predicted ratings
for prediction in predictions:
    true_ratings.append(prediction.r_ui)
    pred_ratings.append(prediction.est)

# Calculate precision, recall, F1 score using scikit-learn
precision, recall, fscore, _ = prfs(true_ratings, [int(round(rating)) for rating in pred_ratings], average='weighted')

# Calculate NDCG using scikit-learn
ndcg = NDCG([true_ratings], [pred_ratings])
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", fscore)
print("NDCG:", ndcg)

Precision: 0.5172525182169726
Recall: 0.4512302416492537
F1 score: 0.41964862661842606
NDCG: 0.9929508043419567
